# 3장 리스트와 튜플

효율적인 프로그램을 작성하려면 자료구조에 대해 이해해야한다.
**배열**(정해진 순서에 따라 데이터를 나열한 것)의 두 가지 자료구조가 있다.
* 리스트: 저장하는 데이터, 배열의 크기를 변경할 수 있는 동적 데이터
* 튜플: 고정된 변경 불가능한 정적 배열 (immutable) -> 런타임에 캐시를 사용해 사용할 때마다 커널에 메모리를 요청하지 않아도 된다.

In [ ]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

Python version
3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
Version info.
sys.version_info(major=3, minor=7, micro=11, releaselevel='final', serial=0)


---
## 배열 생성

주소값이 저장된다.


In [ ]:
# 배열 생성: 시스템 메모리 블록 할당, 블록에는 실제 데이터의 "주소 포인터 값"(integer)이 저장됨

%timeit l = list(range(10))

The slowest run took 8.08 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 417 ns per loop


In [ ]:
%timeit l = list(range(10000000))

1 loop, best of 5: 281 ms per loop


---
## 배열 탐색

선형 탐색보다 더 빠른 탐색이 가능하다

In [ ]:
# 선형탐색: 시간복잡도 O(N)

def linear_search(needle, array):
  for idx, item in enumerate(array):
    if item == needle:
      return idx
  return -1

In [ ]:
# 정렬 후 이분탐색 O(NlogN) ~O(N^2LogN)
# Tim sorting (팀정렬): 최적O(N) 최악 O(NlogN) 어떤 알고리즘을 적용하는 것이 가장 좋은지 최적의 휴리스틱을 찾는다. 삽입정렬과 병합정렬을 조합해 사용

import timeit

def binary_search(needle, haystack):
    # imin and imax store the bounds of the haystack that we are currently
    # considering.  This starts as the bounds of the haystack and slowly
    # converges to surround the needle.
    imin, imax = 0, len(haystack)
    while True:
        if imin >= imax:
            return -1
        midpoint = (imin + imax) // 2
        if haystack[midpoint] > needle:
            imax = midpoint
        elif haystack[midpoint] < needle:
            imin = midpoint + 1
        else:
            return midpoint


if __name__ == "__main__":
    setup = "from __main__ import (binary_search, haystack, needle)"
    iterations = 10000

    for haystack_size in (10000, 100000, 1000000):
        haystack = range(haystack_size)
        for needle in (1, 6000, 9000, 1000000):
            index = binary_search(needle, haystack)
            t = timeit.timeit(
                stmt="binary_search(needle, haystack)", setup=setup, number=iterations
            )
            print(
                f"Value {needle: <8} found in haystack of "
                f"size {len(haystack): <8} at index "
                f"{index: <8} in {t/iterations:.5e} seconds"
            )

In [ ]:
# 파이썬 표준라이브러리 bisect
# 이진탐색 + 새로 삽잆 시에도 정렬 상태 유지

import bisect
import random

def find_closest(haystack, needle):
    # bisect.bisect_left will return the first value in the haystack
    # that is greater than the needle
    i = bisect.bisect_left(haystack, needle)
    if i == len(haystack):
        return i - 1
    elif haystack[i] == needle:
        return i
    elif i > 0:
        j = i - 1
        # since we know the value is larger than needle (and vice versa for the
        # value at j), we don't need to use absolute values here
        if haystack[i] - needle > needle - haystack[j]:
            return j
    return i


if __name__ == "__main__":
    important_numbers = []
    for i in range(10):
        new_number = random.randint(0, 1000)
        bisect.insort(important_numbers, new_number)

    # important_numbers will already be in order because we inserted new elements
    # with bisect.insort
    print(important_numbers)
    # > [14, 265, 496, 661, 683, 734, 881, 892, 973, 992]

    closest_index = find_closest(important_numbers, -250)
    print(f"Closest value to -250: {important_numbers[closest_index]}")
    # > Closest value to -250: 14

    closest_index = find_closest(important_numbers, 500)
    print(f"Closest value to 500: {important_numbers[closest_index]}")
    # > Closest value to 500: 496

    closest_index = find_closest(important_numbers, 1100)
    print(f"Closest value to 1100: {important_numbers[closest_index]}")
    # > Closest value to 1100: 992


[78, 92, 255, 287, 357, 574, 578, 620, 666, 953]
Closest value to -250: 78
Closest value to 500: 574
Closest value to 1100: 953


---
## 리스트의 동적배열

**append()를 지원하며, 이 매서드를 사용하면 새로 메모리를 할당하면서 필요한 이상으로 메모리 블록을 할당한다.**

fn() 과 fn2()를 비교해보면
문장을 추가로 실행해야하고 메모리도 재할당해야하므로 전체 시간도 느리다. ( 9.7 ms : 13.9 ) 1.43배
메모리도 더 필요하다 append를 이용해 1천 size 메모리를 만들면 실제로는 1천 6백만개에 해당하는 메모리를 할당한다. (0.391 MiB : 2.734 MiB)

* peak memory:  the maximum amount of memory your process has used

In [ ]:
# M=(N>>3) + (3 if N<9 else 6)

---
append 와 리스트 내포 메모리 사용량과 실행시간 차이 비교

In [ ]:
# run with $ py -m memory_profiler file.py 

def fn():
    lis = [i*i for i in range(100000)]
    return lis


if __name__ == "__main__":
    fn()

In [ ]:
%memit fn()

UsageError: Line magic function `%memit` not found.



```
Line #    Mem usage    Increment  Occurences   Line Contents
============================================================
     3   40.309 MiB   40.309 MiB           1   @profile
     4                                         def fn1():
     5   44.461 MiB    0.391 MiB      100003       [i*i for i in range(100000)]
     6   40.699 MiB   -3.762 MiB           1       return

```

In [ ]:
%timeit fn()

100 loops, best of 5: 9.7 ms per loop


In [ ]:
def fn2():
    lis = []
    for i in range(100000):
        lis.append(i*2)
    return lis

if __name__ == "__main__":
    fn2()

In [ ]:
%memit fn2()

UsageError: Line magic function `%memit` not found.


```
Line #    Mem usage    Increment  Occurences   Line Contents
============================================================
     8   40.699 MiB   40.699 MiB           1   @profile
     9                                         def fn2():
    10   40.699 MiB    0.000 MiB           1       lis = []
    11   44.445 MiB    2.734 MiB      100001       for i in range(100000):
    12   44.445 MiB    1.012 MiB      100000           lis.append(i*2)
    13   44.445 MiB    0.000 MiB           1       return
```

In [ ]:
%timeit fn2()

100 loops, best of 5: 13.9 ms per loop


---
## 튜플: 정적배열

**append() 지원하지 않고 생성된 후에 변경되지 않는다.**

In [ ]:
# append를 지원하지 않고 새로운 튜플을 생성한다

t1=(1,2,3,4)
t2=(1,2,3,4)
t3=t1+t2

t3

(1, 2, 3, 4, 1, 2, 3, 4)

크기 20 이하인 튜플은 GC 대상이 되지 않아 리소스를 캐싱할 수 있다.
즉 크기 20개 이하의 튜플은 운영체제를 통하지 않으므로 훨씬 빠르게 생성된다.

In [ ]:
%timeit lis = [0,1,2,3,4,5,6,7,8,9]

The slowest run took 8.91 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 119 ns per loop


In [ ]:
%timeit t = (0,1,2,3,4,5,6,7,8,9)

The slowest run took 51.80 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 22.5 ns per loop
